### Agent RAG

In [40]:
! pip install -qU langchain-openai langchain langchain_community langchainhub
! pip install chromadb==0.5.3


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [41]:
from langchain import hub as langchain_hub
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.prompts import PromptTemplate
from string import Template
import uuid

In [42]:
from langchain_openai import OpenAI

api_key = "sk-xxxxx"
api_base = "https://sg.uiuiapi.com/v1"
client = OpenAI(api_key=api_key, base_url=api_base)

In [43]:
# 读取 ./data/data.md 文件作为知识库
file_path = os.path.join('data', 'data.md')
with open(file_path, 'r', encoding='utf-8') as f:
    docs_string = f.read()

# split the document into chunks base on markdown headers
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = text_splitter.split_text(docs_string)
print("Length of splits: ", len(splits))
print(splits)

Length of splits:  14
[Document(metadata={'Header 1': '运维测试知识库', 'Header 2': '目录'}, page_content='- [什么是运维测试](#什么是运维测试)\n- [运维测试的重要性](#运维测试的重要性)\n- [运维测试的类型](#运维测试的类型)\n- [功能测试](#功能测试)\n- [性能测试](#性能测试)\n- [安全测试](#安全测试)\n- [兼容性测试](#兼容性测试)\n- [运维测试的工具](#运维测试的工具)\n- [监控工具](#监控工具)\n- [负载测试工具](#负载测试工具)\n- [安全扫描工具](#安全扫描工具)\n- [运维测试流程](#运维测试流程)\n- [最佳实践](#最佳实践)\n- [参考资料](#参考资料)'), Document(metadata={'Header 1': '运维测试知识库', 'Header 2': '什么是运维测试'}, page_content='运维测试是指在软件开发和运维过程中，通过各种测试手段与工具，对系统进行全面评估，以确保系统的稳定性、性能、安全性和可用性。运维测试贯穿于应用程序的整个生命周期，并不仅仅局限于上线前的测试。'), Document(metadata={'Header 1': '运维测试知识库', 'Header 2': '运维测试的重要性'}, page_content='1. **保障系统稳定性**：通过测试发现潜在问题，减少系统故障率，提高系统的可靠性。\n2. **优化性能**：确认系统在高负载情况下的表现，避免因性能问题导致用户体验下降。\n3. **提升安全性**：及早识别安全漏洞，增强系统的安全防护能力。\n4. **合规性检查**：确保系统符合相关法规和政策要求，降低法律风险。'), Document(metadata={'Header 1': '运维测试知识库', 'Header 2': '运维测试的类型', 'Header 3': '功能测试'}, page_content='验证系统是否按照需求正常工作，包括接口测试、用户界面测试等。'), Document(metadata={'Header 1': '运维测试知识库', 'Header 2': '运维测试的类型'

In [44]:
# 向量化
# 保存到随机目录里
random_directory = "./" + str(uuid.uuid4())
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
# embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory=random_directory)
vectorstore.persist()

In [46]:
# 传统 rag 流程
retriever = vectorstore.as_retriever()

# 提示语模板
template = """使用以下上下文来回答最后的问题。
如果你不知道答案，就说不知道，不要试图编造答案。
最多使用三句话，并尽可能简洁的回答。
在答案的最后一定要说“谢谢询问！”。

{context}
Question: {question}

Helpful Answer:"""
custom_rag_promot = PromptTemplate.from_template(template=template)


def format_docs(docs):
    print("匹配到知识库片段：\n", "\n\n".join(doc.page_content for doc in docs))


llm = ChatOpenAI(model="gpt-4o")
rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | custom_rag_promot
        | llm
        | StrOutputParser()
)

# 传统 RAG 无法回答的问题
res = rag_chain.invoke("谁管的系统最多")
print("\n\nLLM 回答：", res)

匹配到知识库片段：
 小王，管理 payment 系统
小王，管理 payment-1 系统
小李，管理 inventory 系统
小张，管理 order 系统

1. **保障系统稳定性**：通过测试发现潜在问题，减少系统故障率，提高系统的可靠性。
2. **优化性能**：确认系统在高负载情况下的表现，避免因性能问题导致用户体验下降。
3. **提升安全性**：及早识别安全漏洞，增强系统的安全防护能力。
4. **合规性检查**：确保系统符合相关法规和政策要求，降低法律风险。

检查系统的安全性，识别潜在的安全漏洞，确保数据的机密性和完整性。

验证系统是否按照需求正常工作，包括接口测试、用户界面测试等。


LLM 回答： 不知道，因为没有提供具体的上下文和信息来判断哪个系统是由谁管理的。谢谢询问！


In [ ]:
# Agent RAG
# 文本相似性检索
def search_docs(query, k=1):
    result = vectorstore.similarity_search_with_score(query, k=k)
    return "\n\n".join(doc.page_content for doc, score in result)


user_query = "谁管的系统最多？"

check_can_answer_system_prompt = """
根据上下文识别问题是否能够回答，如果不能，则返回 JSON 字符串 "{"can_answer":false}"，如果可以返回"{"can_answer":true}"，
上下文：\n $context
问题： $question
"""

# llm = ChatOpenAI(model="gpt-4o")
k = 1
docs = ""

while True:
    # 检索相关文章，每次循环都会增加一个文档块，最大 15 块
    print("第", k, "次检索")
    if k > 15:
        break
    docs = search_docs(user_query, k)
    print("匹配到的文档是：", docs)
    template = Template(check_can_answer_system_prompt)
    filled_prompt = template.substitute(context=docs, question=user_query)
    # 组装 message
    message = [
        (
            "system", filled_prompt
        ),
        (
            "human", "开始检查上下文是否足够回答问题。"
        )
    ]
    llm_message = llm.invoke(message)
    content = llm_message.content
    print("LLM Res: ", content, "\n")
    if content == '{"can_answer":true}':
        break
    else:
        k += 1
print("已经匹配到能够回答问题的知识库，开始回答\n")

# 最终推理
final_system_prompt = """
您是问答任务的助手，使用检索到的上下文来回答用户提出的问题。如果你不知道答案，就说不知道，不要试图编造答案。
最多使用三句话，并尽可能简洁的回答。
"""

final_messages = [
    (
        "system", final_system_prompt
    ),
    (
        "human", "上下文是:\n" + docs + "\n 问题：" + user_query
    )
]

llm_message = llm.invoke(final_messages)
content = llm_message.content
print("LLM Final Res: ", content, "\n")